In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options
import numpy as np

service = QiskitRuntimeService()

kawasaki = "ibm_kawasaki"
simulator = "ibmq_qasm_simulator"

In [10]:
from qiskit.algorithms import MinimumEigensolverResult


class VQEClientFixed:
    def __init__(self, ansatz, optimizer, service, backend):
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.service = service
        self.backend = backend

    def compute_minimum_eigenvalue(self, hamiltonian, aux_operators=None):
        estimator = Estimator()
        history = []

        with Session(service=self.service, backend=self.simulator):

            def evaluate(x):
                x = list(x)
                results = (
                    estimator.run(
                        circuits=self.ansatz,
                        observables=hamiltonian,
                        parameter_values=[x],
                    )
                    .result()
                    .valuves[0]
                )
                return np.real(results)

            init = np.random.uniform(-np.pi, np.pi, len(self.ansatz.parameters))
            result = self.optimizer.minimize(evaluate, x0=init)

            sol = MinimumEigensolverResult()
            sol.eigenvalue = result.fun

        return sol

In [14]:
import sys

sys.path.append("/work/gs54/s54005/qrao_experiment/qulacs")

from maxcut_instance_generator import _generate_model_from_graph
from networkx import node_link_graph
import pickle

from qrao.encoding import QuantumRandomAccessEncoding
from qrao.quantum_random_access_optimizer import QuantumRandomAccessOptimizer
from qrao.magic_rounding import MagicRounding

from qiskit_optimization.translators import from_docplex_mp

with open(
    "/work/gs54/s54005/qrao_experiment/results2/regular/deg3/nodes30/trial0/graph_data.pkl",
    "rb",
) as f:
    data = pickle.load(f)
graph = node_link_graph(data)
num_nodes = graph.number_of_nodes()
edge_weights = np.zeros((num_nodes, num_nodes))
for i, j in graph.edges():
    weight = 1
    edge_weights[i, j] = edge_weights[j, i] = weight
model = _generate_model_from_graph(graph, edge_weights, num_nodes)
problem = from_docplex_mp(model)

encoding = QuantumRandomAccessEncoding(max_vars_per_qubit=3)
encoding.encode(problem)

print(encoding.num_qubits)

11


In [16]:
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import NFT
from qiskit.circuit.library import EfficientSU2

sweeps = 5
level = 2
entanglement = "linear"
runtime_vqe = VQEClientFixed(
    ansatz=EfficientSU2(
        encoding.num_qubits,
        reps=level,
        entanglement=entanglement,
    ),
    optimizer=NFT(maxfev=(sweeps + 1) * encoding.num_qubits * 4 * (level + 1)),
    service=service,
    backend=simulator,
)

In [18]:
qrao = QuantumRandomAccessOptimizer(
    encoding=encoding,
    min_eigen_solver=runtime_vqe,
    rounding_scheme=None,
)

result, _ = qrao.solve_relaxed()

AttributeError: 'VQEClientFixed' object has no attribute 'supports_aux_operators'